In [3]:
!pip install dash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.0
    Uninstalling Flask-3.1.0:
      Successfully uninstalled Flask-3.1.0


In [10]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Завантаження даних
# Використовуємо прямі посилання на репозиторії з даними
url_confirmed = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
url_deaths = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
url_recovered = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
url_vaccinations = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv'

# Функція для завантаження та попередньої обробки даних
def load_data():
    # Завантаження даних про підтверджені випадки, смерті та одужання
    df_confirmed = pd.read_csv(url_confirmed)
    df_deaths = pd.read_csv(url_deaths)
    df_recovered = pd.read_csv(url_recovered)

    # Підготовка даних для аналізу
    # Перетворення з широкого формату в довгий
    dates_confirmed = df_confirmed.columns[4:]

    # Підготовка даних про підтверджені випадки
    confirmed_long = pd.melt(
        df_confirmed,
        id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
        value_vars=dates_confirmed,
        var_name='date',
        value_name='confirmed'
    )

    # Підготовка даних про смерті
    deaths_long = pd.melt(
        df_deaths,
        id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
        value_vars=dates_confirmed,
        var_name='date',
        value_name='deaths'
    )

    # Об'єднання даних
    covid_data = confirmed_long.merge(
        deaths_long,
        on=['Province/State', 'Country/Region', 'Lat', 'Long', 'date']
    )

    # Перетворення дати в правильний формат
    covid_data['date'] = pd.to_datetime(covid_data['date'])

    # Агрегація даних по країнах
    country_data = covid_data.groupby(['Country/Region', 'date']).agg({
        'confirmed': 'sum',
        'deaths': 'sum'
    }).reset_index()

    # Обчислення нових випадків і смертей
    country_data = country_data.sort_values(['Country/Region', 'date'])
    country_data['new_cases'] = country_data.groupby('Country/Region')['confirmed'].diff().fillna(0)
    country_data['new_deaths'] = country_data.groupby('Country/Region')['deaths'].diff().fillna(0)

    # Виправлення негативних значень
    country_data['new_cases'] = country_data['new_cases'].clip(lower=0)
    country_data['new_deaths'] = country_data['new_deaths'].clip(lower=0)

    # Обчислення 7-денних рухомих середніх
    country_data['new_cases_7day_avg'] = country_data.groupby('Country/Region')['new_cases'].rolling(7).mean().reset_index(0, drop=True)
    country_data['new_deaths_7day_avg'] = country_data.groupby('Country/Region')['new_deaths'].rolling(7).mean().reset_index(0, drop=True)

    # Завантаження даних про вакцинацію
    try:
        df_vaccinations = pd.read_csv(url_vaccinations)
        df_vaccinations['date'] = pd.to_datetime(df_vaccinations['date'])

        # Вибір найбільш важливих колонок
        vac_cols = ['location', 'date', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred']
        vac_data = df_vaccinations[vac_cols].copy()

        # Перейменування для узгодження з нашими даними
        vac_data = vac_data.rename(columns={'location': 'Country/Region'})

        # Об'єднання даних про випадки і вакцинацію
        combined_data = pd.merge(
            country_data,
            vac_data,
            on=['Country/Region', 'date'],
            how='left'
        )
    except Exception as e:
        print(f"Помилка при завантаженні даних вакцинації: {e}")
        combined_data = country_data

    return combined_data

# Завантаження даних
df = load_data()

# Список країн для вибору
countries = sorted(df['Country/Region'].unique())

# Ініціалізація додатку Dash
app = dash.Dash(__name__, meta_tags=[{"name": "viewport", "content": "width=device-width"}])
server = app.server

# Заголовок додатку
app.title = "COVID-19 Дашборд"

# Створення макету дашборду
app.layout = html.Div([
    # Заголовок
    html.Div([
        html.H1("COVID-19 Глобальний Дашборд", style={"text-align": "center"}),
        html.P("Аналіз поширення COVID-19, смертності та вакцинації у світі",
               style={"text-align": "center", "font-size": "1.2em"}),
    ], className="header"),

    # Вибір країн і періоду часу
    html.Div([
        html.Div([
            html.Label("Виберіть країни:"),
            dcc.Dropdown(
                id='country-dropdown',
                options=[{'label': country, 'value': country} for country in countries],
                value=['US', 'Ukraine', 'Italy', 'Germany', 'India'],
                multi=True
            ),
        ], className="six columns"),

        html.Div([
            html.Label("Виберіть період:"),
            dcc.DatePickerRange(
                id='date-picker',
                min_date_allowed=df['date'].min(),
                max_date_allowed=df['date'].max(),
                start_date=df['date'].max() - pd.DateOffset(months=6),
                end_date=df['date'].max(),
                display_format='YYYY-MM-DD'
            ),
        ], className="six columns"),
    ], className="row filters"),

    # Графіки для підтверджених випадків
    html.Div([
        html.H2("Динаміка підтверджених випадків COVID-19"),
        dcc.Graph(id='cases-graph'),
    ], className="row graph-container"),

    # Графіки для смертей
    html.Div([
        html.H2("Динаміка смертей від COVID-19"),
        dcc.Graph(id='deaths-graph'),
    ], className="row graph-container"),

    # Дані про вакцинацію
    html.Div([
        html.H2("Рівень вакцинації"),
        dcc.Graph(id='vac-graph'),
    ], className="row graph-container"),

    # Порівняння країн
    html.Div([
        html.H2("Порівняння країн"),
        dcc.Graph(id='country-comparison'),
    ], className="row graph-container"),

    # Інформаційний розділ
    html.Div([
        html.H2("Аналіз даних COVID-19"),
        html.P([
            "Цей дашборд показує динаміку поширення COVID-19, смертності та вакцинації в різних країнах світу. ",
            "Дані оновлюються регулярно і отримані з репозиторію ",
            html.A("Johns Hopkins University CSSE",
                  href="https://github.com/CSSEGISandData/COVID-19",
                  target="_blank"),
            " і бази даних вакцинації від ",
            html.A("Our World in Data",
                  href="https://github.com/owid/covid-19-data",
                  target="_blank"),
            "."
        ]),
    ], className="row", style={"margin-top": "20px"}),

    # Футер
    html.Footer([
        html.P("Дані актуальні на: " + str(df['date'].max().strftime('%Y-%m-%d')),
              style={"text-align": "center"})
    ])
])

# Зворотні виклики для оновлення графіків

# Графік підтверджених випадків
@app.callback(
    Output('cases-graph', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_cases_graph(selected_countries, start_date, end_date):
    filtered_df = df[
        (df['Country/Region'].isin(selected_countries)) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]

    fig = px.line(
        filtered_df,
        x='date',
        y='new_cases_7day_avg',
        color='Country/Region',
        title='Нові випадки COVID-19 (7-денне середнє)',
        labels={'new_cases_7day_avg': 'Нові випадки (7-денне середнє)', 'date': 'Дата', 'Country/Region': 'Країна'}
    )

    fig.update_layout(
        xaxis_title='Дата',
        yaxis_title='Нові випадки (7-денне середнє)',
        legend_title='Країна',
        hovermode='x unified'
    )

    return fig

# Графік смертей
@app.callback(
    Output('deaths-graph', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_deaths_graph(selected_countries, start_date, end_date):
    filtered_df = df[
        (df['Country/Region'].isin(selected_countries)) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]

    fig = px.line(
        filtered_df,
        x='date',
        y='new_deaths_7day_avg',
        color='Country/Region',
        title='Нові смерті від COVID-19 (7-денне середнє)',
        labels={'new_deaths_7day_avg': 'Нові смерті (7-денне середнє)', 'date': 'Дата', 'Country/Region': 'Країна'}
    )

    fig.update_layout(
        xaxis_title='Дата',
        yaxis_title='Нові смерті (7-денне середнє)',
        legend_title='Країна',
        hovermode='x unified'
    )

    return fig

# Графік вакцинації
@app.callback(
    Output('vac-graph', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_vac_graph(selected_countries, start_date, end_date):
    filtered_df = df[
        (df['Country/Region'].isin(selected_countries)) &
        (df['date'] >= start_date) &
        (df['date'] <= end_date)
    ]

    if 'people_fully_vaccinated_per_hundred' in filtered_df.columns:
        fig = px.line(
            filtered_df,
            x='date',
            y='people_fully_vaccinated_per_hundred',
            color='Country/Region',
            title='Повна вакцинація (% населення)',
            labels={'people_fully_vaccinated_per_hundred': 'Повністю вакциновані (%)',
                   'date': 'Дата',
                   'Country/Region': 'Країна'}
        )

        fig.update_layout(
            xaxis_title='Дата',
            yaxis_title='Повністю вакциновані (%)',
            legend_title='Країна',
            hovermode='x unified'
        )
    else:
        fig = go.Figure()
        fig.update_layout(
            title='Дані про вакцинацію недоступні',
            xaxis_title='Дата',
            yaxis_title='Вакцинація (%)',
            annotations=[{"text": "Дані про вакцинацію недоступні", "showarrow": False, "font": {"size": 20}}]
        )

    return fig

# Порівняння країн
@app.callback(
    Output('country-comparison', 'figure'),
    [Input('country-dropdown', 'value'),
     Input('date-picker', 'end_date')]
)
def update_country_comparison(selected_countries, end_date):
    # Беремо останню доступну дату перед або рівну end_date
    latest_date = df[df['date'] <= end_date]['date'].max()

    # Фільтруємо дані
    filtered_df = df[
        (df['Country/Region'].isin(selected_countries)) &
        (df['date'] == latest_date)
    ]

    # Створюємо порівняльну діаграму
    fig = px.bar(
        filtered_df,
        x='Country/Region',
        y='deaths',
        color='Country/Region',
        title=f'Загальна кількість смертей на {latest_date.strftime("%Y-%m-%d")}',
        labels={'deaths': 'Загальна кількість смертей', 'Country/Region': 'Країна'}
    )

    # Додаємо другу вісь з кількістю підтверджених випадків
    fig.add_trace(
        go.Scatter(
            x=filtered_df['Country/Region'],
            y=filtered_df['confirmed'],
            mode='markers',
            marker=dict(size=15, color='red'),
            name='Загальна кількість випадків',
            yaxis='y2'
        )
    )

    # Налаштування макету
    fig.update_layout(
        xaxis_title='Країна',
        yaxis_title='Загальна кількість смертей',
        yaxis2=dict(
            title='Загальна кількість випадків',
            titlefont=dict(color='red'),
            tickfont=dict(color='red'),
            anchor='x',
            overlaying='y',
            side='right'
        ),
        legend_title='Країна',
        hovermode='closest'
    )

    return fig

# Запуск сервера
if __name__ == '__main__':
    app.run(debug=True)

<ipython-input-10-faa53712524e>:52: UserWarning:

Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.



<IPython.core.display.Javascript object>